In [1]:
from ga_regression import *
import pandas as pd

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
file = Path(r"D:\resynth\run_38_39\run00038_resynth\2025-07-31-13-38-23\opts_and_metadata.pt")  # no decay, try lr
readers = Readers.from_file(file)
reader = readers[0]
m = reader.metadata

In [3]:
stim = Path(r"D:\resynth\temp\mesh_evo_run00015_gen0000_lin00\mesh_evo_run00015_gen0000_lin00.hdf")
df = pd.read_hdf(stim, 'scenes')

In [14]:
m.channel

(14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 9, 20, 11, 18)

In [44]:
df_scenes = GaDataset.load_scenes(data_file=stim, file_mode=m.opts.mesh_file_mode, n_faces=m.n_faces, features=m.input_features)
df_responses = pd.DataFrame(
    np.zeros((len(df_scenes), len(m.channel))),
    index=df_scenes.index,
    columns=pd.Index(m.channel, name='channel')
)
df_responses

channel,14,17,29,23,2,0,13,31,3,26,9,20,11,18
scene,,,,,,,,,,,,,,
49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_responses.columns.name = 'what'
df_responses.columns

Index([14, 17, 29, 23, 2, 0, 13, 31, 3, 26, 9, 20, 11, 18], dtype='int64', name='what')

In [17]:
Path(stim.parent / df.simplified.iloc[0]).exists()
Path(stim.parent / 'op_cache').mkdir(exist_ok=True)

In [19]:
dataset = GaDataset(
    df=df_scenes,
    responses=df_responses,
    root_dir=stim.parent,
    k_eig=m.k_eig,
    op_cache_dir=stim.parent / 'op_cache',
    file_mode=m.opts.mesh_file_mode,
    weights=None,
    use_visible=m.use_visible,
    use_color=m.use_color,
    norm_verts=m.norm_verts,
    features=m.input_features,
    augment=None,
)

In [29]:
loader = DataLoader(dataset, shuffle=False, batch_size=None)
expt = reader.experiment()

In [30]:
_obs, preds = expt.predict(loader, agg_fn=np.stack)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.79it/s]


In [32]:
preds.shape  # n_stim, n_ch

(2, 14)